In [1]:
import time
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from time import sleep

In [2]:
db = pd.read_csv('base_de_datos.csv', dtype = {'Sorteos': "string"}, parse_dates = ['Dates'])

In [3]:
winners = db.iloc[:, 2:7]

In [4]:
for e in range(1,6):
    for i in range(1,51):
        winners.loc[winners[f"Nro{e}"] == i, f"{i}"] = True

In [5]:
winners.drop(columns = ["Nro1","Nro2","Nro3","Nro4","Nro5"], inplace = True)

In [6]:
# for columns in winners:
#     if winners.loc[winners['1']:-1] == True:
#         winners.isna().sum()

In [7]:
# values = list(set(val for dic in [ultimo_sorteo] for val in dic.values()))

In [8]:
draws = []
for i in range(1, 13):
    if i != 13:
        rows = len(winners) - i
        draws.append(rows)

c = [str(i) for i in range(1, 51)]
dicts = {draw: {key:[] for key in c} for draw in draws}

In [9]:
contador = 0
e = 0
while e != 12:
    for columns in winners:
        winners = winners.loc[:draws[e]]
        for i in reversed(winners[columns]):
            if i is not True:
                contador += 1
            else:
                dicts[draws[e]].update({columns:contador})
                contador = 0
                break
    e += 1

In [10]:
dicts

{1602: {'1': 12,
  '2': 13,
  '3': 3,
  '4': 5,
  '5': 24,
  '6': 3,
  '7': 0,
  '8': 1,
  '9': 1,
  '10': 29,
  '11': 2,
  '12': 0,
  '13': 3,
  '14': 27,
  '15': 1,
  '16': 7,
  '17': 9,
  '18': 4,
  '19': 0,
  '20': 8,
  '21': 4,
  '22': 20,
  '23': 11,
  '24': 18,
  '25': 5,
  '26': 11,
  '27': 2,
  '28': 27,
  '29': 8,
  '30': 4,
  '31': 1,
  '32': 24,
  '33': 0,
  '34': 6,
  '35': 7,
  '36': 39,
  '37': 5,
  '38': 18,
  '39': 17,
  '40': 3,
  '41': 2,
  '42': 3,
  '43': 0,
  '44': 7,
  '45': 7,
  '46': 4,
  '47': 2,
  '48': 9,
  '49': 39,
  '50': 4},
 1601: {'1': 11,
  '2': 12,
  '3': 2,
  '4': 4,
  '5': 23,
  '6': 2,
  '7': 27,
  '8': 0,
  '9': 0,
  '10': 28,
  '11': 1,
  '12': 13,
  '13': 2,
  '14': 26,
  '15': 0,
  '16': 6,
  '17': 8,
  '18': 3,
  '19': 0,
  '20': 7,
  '21': 3,
  '22': 19,
  '23': 10,
  '24': 17,
  '25': 4,
  '26': 10,
  '27': 1,
  '28': 26,
  '29': 7,
  '30': 3,
  '31': 0,
  '32': 23,
  '33': 4,
  '34': 5,
  '35': 6,
  '36': 38,
  '37': 4,
  '38': 17,
  '39':